In [1]:
import pickle
import glob
import pandas as pd
import numpy as np

In [2]:
path = './pickled/13일17일'

file_list = glob.glob(path + "\\*.pkl")
df_list = []

for file in file_list:
    current_data = pd.read_pickle(file)
    df_list.append(current_data)

In [3]:
len(df_list)

5

In [21]:
# dat01 = df_list[0] #8
# train02 = dat01.copy()
# dat02 = df_list[1] #9
# train02 = dat02.copy()
# dat03 = df_list[2] #10
# train02 = dat03.copy()
# dat04 = df_list[3] #11
# train02 = dat04.copy()
# dat05 = df_list[4] #12
# train02 = dat05.copy()
# dat06 = df_list[0] #13
# train02 = dat06.copy()
# dat07 = df_list[1] #14
# train02 = dat07.copy()
# dat08 = df_list[2] #15
# train02 = dat08.copy()
# dat09 = df_list[3] #16
# train02 = dat09.copy()
dat10 = df_list[4] #17
train02 = dat10.copy()

# pivot table 생성

In [22]:
# pivot_table
train02['count'] = 1
train02['log_id'].astype(str)
train03 = train02.pivot_table(values = 'count', index = 'actor_account', columns = 'log_id', aggfunc = sum, fill_value = 0)
train03.rename({'account':'actor_account'}, axis = 1)
train04 = train03.copy()

# 변수생성

In [23]:
day = 'day17'

In [24]:
header = train04.columns.astype(str)
train04.columns = header
train05 = train04.copy()
train05['103'] = train05['103'].replace(0, 1)
train05['160'] = train05['160'].replace(0, 1)
# 모든 로그 카운트(log_sum)
train05['log_sum']=train05.sum(axis=1) 
# 계정당 접속 ip 확인(ip_count)
ip01 = train02[train02['log_id'] == 103]
ip02 = ip01[['actor_account','etc_str1']]
ip03 = ip02.drop_duplicates(subset = ['etc_str1'])
ip04 = ip03.groupby('actor_account').count()
ip04.rename({'etc_str1':'log_count'}, axis = 1, inplace = True)
train05 = pd.merge(train05, ip04, on='actor_account', how='left')
train05['log_count'] = train05['log_count'].fillna(1)
train05['log_ip_ratio'] = (train05['103'] / train05['log_count']).round(4)
# 접속일 계산(Max=10)
da01 = train02[['actor_account', 'log_date']]
da01['for_count'] = da01['log_date'].str[0:10]
da01_gp = pd.DataFrame(data=da01.groupby('actor_account')['for_count'].nunique())
da01_gp['for_count'] = 1
da01_gp.rename(columns={"for_count":"per_day"},inplace=True)
train05 = pd.merge(train05, da01_gp,left_on=train05.index,right_on=da01_gp.index, how='left',left_index=True, right_index=True)
train05['log_ip_perday'] = (train05['103'] / train05['per_day']).round(4)
# 전체 플레이시간
train05.rename({'160':'play_time'},axis = 1, inplace = True)
# train05['play_time'].fillna(1)
# quiz 관련 변수
train05['quiz_per_playtime'] = (train05['116'] / train05['play_time']).round(4)
train05['quiz_per_logcount'] = (train05['116'] / train05['log_sum']).round(4)
# 평균 플레이시간
train05['ply_time_perlog'] = (train05['play_time'] / train05['103']).round(4)
train05['ply_time_perday'] = (train05['play_time'] / train05['per_day']).round(4)
# 최고레벨 max_level_of_a_day => 합치면서 10일치 확인
lv01 = train02[train02['log_id'] == 104]
lv02 = lv01[['actor','actor_account','etc_num4']]
lv03 = lv02.sort_values(['actor'], ascending = False)
lv04 = lv03.groupby('actor').max()
lv05 = lv04.groupby('actor_account').max()
lv05.rename(columns={"etc_num4":"max_level_of_a_day" + day}, inplace=True)
train05 = pd.merge(train05, lv05, how='left', left_on=train05.index, right_on=lv05.index, right_index=True)
# 앉은 횟수 비율
train05['sit_per_time'] = (train05['118'] / train05['play_time']).round(4)
# 전체 친구 수
friends01 = train02[train02['log_id'] == 134]
friends02 = friends01.drop_duplicates(['actor_account'],keep="last")
friends02.set_index(friends02['actor_account'], inplace = True)
friends03 = friends02[['etc_num10']]
train05 = pd.merge(train05, friends03, how='left',left_on=train05.index, right_on=friends03.index,left_index=True,right_index=True)
train05.rename(columns={"etc_num10":"friends_count"},inplace=True)
train05['friends_count'] = train05['friends_count'].fillna(0)
# 총 플레이어 킬 횟수
player_kill_count = train02[train02['log_id'] == 137]
player_kill_count = player_kill_count['target_account'].value_counts()
train05 = pd.merge(train05, player_kill_count,left_on=train05.index,right_on=player_kill_count.index, how='left',left_index=True,right_index=True)
train05.rename(columns={"target_account":"player_kill_count"},inplace=True)
train05['player_kill_count'] = train05['player_kill_count'].fillna(0)
# 게임 시간 당 pc에게 죽음, NPC에게 죽음
train05['killed_pc_pertime'] = (train05['137'] / train05['play_time']).round(4)
train05['killed_npc_pertime'] = (train05['138'] / train05['play_time']).round(4)
train05['killed_pc_pertime'] = train05['killed_pc_pertime'].fillna(0)
train05['killed_npc_pertime'] = train05['killed_npc_pertime'].fillna(0)
train05['killed_sum'] = train05['killed_pc_pertime'] + train05['killed_npc_pertime']
# train05.drop(columns = ['137','138'], inplace = True)
# 텔레포트
train05['port_pertime'] = (train05['142'] / train05['play_time']).round(4)
# 경험치(전체, 시간평균, 로그평균)
get_exp_amount = train02[train02['log_id'] == 143]
get_exp_amount['exp_amount'] = get_exp_amount['etc_num6'] - get_exp_amount['etc_num5']
get_exp_amount = get_exp_amount[['actor_account', 'exp_amount']]
get_exp_amount = get_exp_amount.groupby('actor_account').sum()
train05 = pd.merge(train05,get_exp_amount, how='left',left_on=train05.index, right_on=get_exp_amount.index, right_index=True)
train05['exp_pertime'] = (train05['exp_amount'] / train05['play_time']).round(4)
train05['exp_perlog'] = (train05['exp_amount'] / train05['103']).round(4)
train05['exp_amount'] = train05['exp_amount'].fillna(0)
train05['exp_pertime'] = train05['exp_pertime'].fillna(0)
train05['exp_perlog'] = train05['exp_perlog'].fillna(0)
# 시공이용 관련
train05['151_pertime'] = (train05['151'] / train05['play_time']).round(4)
train05['151_perlog'] = (train05['151'] / train05['103']).round(4)
# 얻은 어비스 획득량
df_156_all = train02[train02['log_id']==156]
df_156_all = df_156_all.drop_duplicates(subset='actor', keep='last')
df_156_all = df_156_all.groupby(['actor_account']).sum()
df_156_all['get_AP'] = df_156_all[['etc_num6']]
df_156_all = df_156_all[['get_AP']]
train05 = pd.merge(train05, df_156_all, how='left', left_on=train05.index, right_on=df_156_all.index, right_index=True)
train05['get_AP'] = train05['get_AP'].fillna(0)
train05['getap_pertime'] = (train05['get_AP'] / train05['play_time']).round(4).fillna(0)
train05['getap_perlog'] = (train05['get_AP'] / train05['103']).round(4).fillna(0)
# 누적 어비스 획득량
df_156_all = train02[train02['log_id']==156]
df_156_all = df_156_all.drop_duplicates(subset='actor', keep='last')
df_156_all = df_156_all.groupby(['actor_account']).sum()
df_156_all['total_AP'] = df_156_all[['etc_num6']]
df_156_all = df_156_all[['total_AP']]
train05 = pd.merge(train05, df_156_all, how='left', left_on=train05.index, right_on=df_156_all.index, right_index=True)
train05['total_AP'] = train05['total_AP'].fillna(0)
train05['totalap_pertime'] = (train05['total_AP'] / train05['play_time']).round(4).fillna(0)
train05['totalap_perlog'] = (train05['total_AP'] / train05['103']).round(4).fillna(0)
# 키나 관련 변수 생성
train05['money_get'] = (train05['187'] / train05['play_time']).round(4).fillna(0)
train05['money_use'] = (train05['188'] / train05['play_time']).round(4).fillna(0)
train05['money_get_ratio'] = (train05['187'] / train05['103']).round(4).fillna(0)
train05['money_use_ratio'] = (train05['188'] / train05['103']).round(4).fillna(0)
# 증감 키나량
mo01 = train02[train02['log_id'] == 104]
mo02 = mo01[['actor_account','etc_num1']]
mo03 = mo02.groupby('actor_account').sum()
mo03.rename(columns={"etc_num1":"exp_increasingrate"}, inplace=True)
train05 = pd.merge(train05, mo03, how='left', left_on=train05.index, right_on=mo03.index, right_index=True)
train05['exp_increasingrate'] = train05['exp_pertime'].replace(0, 1)
train05['exp_increasingrate'] = train05['exp_pertime'].fillna(1)
train05['exp_pertime'] = (train05['exp_increasingrate'] / train05['play_time']).round(4).fillna(0)
train05['exp_perlog'] = (train05['exp_increasingrate'] / train05['103']).round(4).fillna(0)
# 아이템 관련
item_get = train02[train02['log_id'] == 202]
item_get = item_get.groupby('actor_account').sum()
item_get = item_get[['etc_num1']]
item_get.rename(columns={'etc_num1':'get_item'},inplace=True)
train05 = pd.merge(train05, item_get, how='left', left_on=train05.index, right_on=item_get.index, right_index=True)
train05['get_item'] = train05['get_item'].replace(0, 1)
train05['get_item'] = train05['get_item'].fillna(1)
train05['item_pertime'] = (train05['get_item'] / train05['play_time']).round(4).fillna(0)
train05['item_perlog'] = (train05['get_item'] / train05['103']).round(4).fillna(0)
train05.to_csv(day + 'TEST.csv',index=True)

<ipython-input-24-9e8650a4393c>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  da01['for_count'] = da01['log_date'].str[0:10]
<ipython-input-24-9e8650a4393c>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  get_exp_amount['exp_amount'] = get_exp_amount['etc_num6'] - get_exp_amount['etc_num5']
